In [ ]:
import argparse
import json
import tensorboard
import tensorboardX
import os
import argparse
import json
import os
import numpy as np
import torch
import torch.nn as nn
import nni
from nni.nas.nn.pytorch import ModelSpace, LayerChoice, MutableConv2d, MutableBatchNorm2d, MutableReLU
from pytorch_lightning import Trainer
from nni.nas.evaluator.pytorch import Lightning, ClassificationModule, Trainer
from nni.nas.experiment import NasExperiment
from nni.nas.space import model_context
from nni.nas.hub.pytorch import DARTS
from nni.nas.strategy import DARTS as DartsStrategy
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from torchvision.datasets import CIFAR10
from nni.nas.experiment import NasExperiment
from nni.nas.evaluator import FunctionalEvaluator
from nni.nas.evaluator import FunctionalEvaluator
import nni.nas.strategy as strategy
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
#from ops import AvgPool,DilConv,SepConv
import genotypes
from pytorch_lightning.callbacks import ModelCheckpoint
torch.set_float32_matmul_precision('medium')
from tqdm import tqdm
from nni.nas.nn.pytorch import LayerChoice, ModelSpace,ValueChoice
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from pytorch_lightning import LightningModule, Trainer
from torchvision import datasets, transforms
from nni.nas.evaluator.pytorch import Classification

In [ ]:

class SepConv(nn.Module):
    def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True):
        super(SepConv, self).__init__()
        self.op = nn.Sequential(
            MutableConv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding),
            MutableConv2d(C_in, C_in, kernel_size=1, padding=0),
            MutableConv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding),
            MutableConv2d(C_in, C_out, kernel_size=1, padding=0),
            MutableBatchNorm2d(C_out,affine=affine),
            MutableReLU()
        )
    
    def forward(self, x):
        return self.op(x)
        
                
        
class PhotonicSigmoid(nn.Module):
    def forward(self, x):
        tmp = torch.exp((x - 0.145) / 0.073)
        tmp = 1.005 + (0.06 - 1.005) / (1 + tmp)
        return tmp.float()


In [ ]:

class CustomDARTSSpaceK3(ModelSpace):
    def __init__(self, input_channels, channels, num_classes, layers, verbose):
        super(CustomDARTSSpaceK3, self).__init__()
        #self.first_iter = True
        self.layers = nn.ModuleList()
        self.drop_path_prob = 0.0  
        self.preliminary_layer = nn.Conv2d(3, 8, kernel_size=3, padding=0, bias=False)
        self.verbose = verbose

        layer1_in = 8
        layer1_out = nni.choice('layer1_out_channels', [2,4,8,16,32,64])
        layer2_out= nni.choice('layer2_out_channels', [2,4,8,16,32,64])
        layer3_out= nni.choice('layer3_out_channels', [2,4,8,16,32,64])
        layer4_out= nni.choice('layer4_out_channels', [2,4,8,16,32,64])
        layer5_out = 22 

        
        layer1 = LayerChoice([
              SepConv(8, layer1_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(8, layer1_out, kernel_size=1),
                MutableBatchNorm2d(layer1_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(8, layer1_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer1_out),
                MutableReLU()
            )
        ], label='layer_1')
        self.layers.append(layer1)
        
        layer2 = LayerChoice([
            SepConv(layer1_out, layer2_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(layer1_out, layer2_out, kernel_size=1),
                MutableBatchNorm2d(layer2_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(layer1_out, layer2_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer2_out),
                MutableReLU()
            )
        ], label='layer_2')
        self.layers.append(layer2)
        
        layer3 = LayerChoice([
            SepConv(layer2_out, layer3_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(layer2_out, layer3_out, kernel_size=1),
                MutableBatchNorm2d(layer3_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(layer2_out, layer3_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer3_out),
                MutableReLU()
            )
        ], label='layer_3')
        self.layers.append(layer3)
        
        layer4 = LayerChoice([
            SepConv(layer3_out, layer4_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(layer3_out, layer4_out, kernel_size=1),
                MutableBatchNorm2d(layer4_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(layer3_out, layer4_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer4_out),
                MutableReLU()
            )
        ], label='layer_4')
        self.layers.append(layer4)
        
        layer5 = LayerChoice([
            SepConv(layer4_out, layer5_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(layer4_out, layer5_out, kernel_size=1),
                MutableBatchNorm2d(layer5_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(layer4_out, layer5_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer5_out),
                MutableReLU()
            )
        ], label='layer_5')
        self.layers.append(layer5)
        
        # Ensure the number of inputs to fc1 is close to but not exceeding 200
        self.pool = nn.AdaptiveAvgPool2d((2, 2))
        self.fc1 = nn.Linear(198, 128) 
        self.fc2 = nn.Linear(128, 64) 
        self.fc3 = nn.Linear(64, 32)  
        self.relu = nn.ReLU()
        self.classifier = nn.Linear(32, num_classes)

    def forward(self, x):
        # Kernel: 2
        # Initial shape: 32x32
        #print(f'Input shape: {x.shape}')
        x = self.preliminary_layer(x)
        # After preliminary layer: 31x31
        if self.verbose == 1 :
            print(f'After preliminary layer: {x.shape}')
        
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if self.verbose == 1 :
                print(f'After layer {i+1}: {x.shape}')
            
            # Add a AvgPool2d layer after the third layer to reduce spatial dimensions
            if i == 2 or i == 4:
                x = nn.AvgPool2d(kernel_size=2, stride=2)(x)
                if self.verbose == 1 :
                    print(f'After avg pooling: {x.shape}')
        

        x =  nn.AvgPool2d(kernel_size=2, stride =2)(x)
        if self.verbose == 1 :
            print(f'After adaptive pooling: {x.shape}')
        
        x = torch.flatten(x, 1)
        if self.verbose == 1 :
            print(f'After flattening: {x.shape}')
        
        x = self.fc1(x)
        x= self.relu(x)
        if self.verbose == 1 :
            print(f'After fc1: {x.shape}')
        x = self.fc2(x)
        x= self.relu(x)
        if self.verbose == 1 :
            print(f'After fc2: {x.shape}')
        x = self.fc3(x)
        x= self.relu(x)
        if self.verbose == 1 :
            print(f'After fc3: {x.shape}')
        
        x = self.classifier(x)
        if self.verbose == 1 :
            print(f'After classifier: {x.shape}')
        #self.first_iter = False
        return x

    def set_drop_path_prob(self, drop_path_prob):
        self.drop_path_prob = drop_path_prob
        for layer in self.layers:
            if hasattr(layer, 'set_drop_path_prob'):
                layer.set_drop_path_prob(drop_path_prob)


In [ ]:
checkpoint_path = './checkpoints/best-checkpoint-v5.ckpt'

checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

model_checkpoint = CustomDARTSSpaceK3(input_channels=3, channels=64, num_classes=10, layers=5,verbose =0)

model_state_dict = model_checkpoint.state_dict()
pretrained_state_dict = checkpoint['state_dict']

filtered_state_dict = {k: v for k, v in pretrained_state_dict.items() if k in model_state_dict}

model_state_dict.update(filtered_state_dict)

model_checkpoint.load_state_dict(model_state_dict)


if 'global_step' in checkpoint:
    print("Global Step:", checkpoint['global_step'])

if 'callbacks' in checkpoint and isinstance(checkpoint['callbacks'], dict):
    for key, callback in checkpoint['callbacks'].items():
        if isinstance(callback, dict) and 'best_model_score' in callback:
            print("Best Model Score (Train Accuracy):", callback['best_model_score'].item())

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_checkpoint = model_checkpoint.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-5)  

if 'optimizer_state_dict' in checkpoint:
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model_checkpoint.train()

def continue_training(model, train_loader, criterion, optimizer, num_epochs, start_epoch=0):
    for epoch in range(start_epoch, num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')

    print('Finished Training')


# Continue training the model
continue_training(model_checkpoint, train_loader, criterion, optimizer, num_epochs=600, start_epoch=0)